In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import datasets
from sklearn.metrics import accuracy_score
import timm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(60),
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
dataset = datasets.ImageFolder(r'C:\Users\Saumya\PycharmProjects\pythonProject6\data\AugmentedAlzheimerDataset',transform=train_transform)


In [5]:
class_counts = {}
subset_indices = []

for idx, (_, label) in enumerate(dataset.samples):  
    if class_counts.get(label, 0) < 4000:  
        subset_indices.append(idx)
        class_counts[label] = class_counts.get(label, 0) + 1

# Create a subset dataset
subset_dataset = Subset(dataset, subset_indices)


train_size = int(0.8 * len(subset_indices))  # 80% train
test_size = len(subset_indices) - train_size  # 20% test

# Split the subset dataset
train_dataset, test_dataset = random_split(subset_dataset, [train_size, test_size])
train_val_split = int(0.8 * train_size)  
val_size = train_size - train_val_split
train_dataset, val_dataset = random_split(train_dataset, [train_val_split, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [6]:
testing_dataset = datasets.ImageFolder(r'C:\Users\Saumya\PycharmProjects\pythonProject6\OriginalDataset',transform=test_transform)

In [7]:
final_test_loader = DataLoader(testing_dataset, batch_size=32, shuffle=False)

In [8]:
class_mapping = {
    "Mild Impairment":"MildDemented",
    "Moderate Impairment":"ModeratedDemented",
    "No Impairment":"NonDemented",
    "Very Mild Impairment":"VeryMildDemented"
}

In [9]:
new_test_dataset = datasets.ImageFolder(r"C:\Users\Saumya\PycharmProjects\pythonProject6\test",transform=test_transform)
new_test_dataset.class_to_idx = {class_mapping[c]: idx for c, idx in new_test_dataset.class_to_idx.items() if c in class_mapping}

In [10]:
new_test_loader = DataLoader(new_test_dataset, batch_size=32, shuffle=False)

In [11]:
from timm import create_model

model = create_model('coatnext_nano_rw_224.sw_in1k', pretrained=True)
print(model)

MaxxVit(
  (stem): Stem(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (norm1): LayerNormAct2d(
      (32,), eps=1e-06, elementwise_affine=True
      (drop): Identity()
      (act): GELU()
    )
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (stages): Sequential(
    (0): MaxxVitStage(
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (shortcut): Downsample2d(
            (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
            (expand): Identity()
          )
          (down): Identity()
          (conv_dw): Conv2d(64, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), groups=64)
          (norm): LayerNorm2d((64,), eps=1e-06, elementwise_affine=True)
          (mlp): ConvMlp(
            (fc1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
            (norm): Identity()
            (act): GELU()
            (drop): Dropout(p=0.0, inplace=False)
         

In [12]:
print(model)

MaxxVit(
  (stem): Stem(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (norm1): LayerNormAct2d(
      (32,), eps=1e-06, elementwise_affine=True
      (drop): Identity()
      (act): GELU()
    )
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (stages): Sequential(
    (0): MaxxVitStage(
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (shortcut): Downsample2d(
            (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
            (expand): Identity()
          )
          (down): Identity()
          (conv_dw): Conv2d(64, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), groups=64)
          (norm): LayerNorm2d((64,), eps=1e-06, elementwise_affine=True)
          (mlp): ConvMlp(
            (fc1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
            (norm): Identity()
            (act): GELU()
            (drop): Dropout(p=0.0, inplace=False)
         

In [13]:
for param in model.parameters():
    param.requires_grad = False
old_head = model.head
num_features = model.head.fc.out_features  
num_classes = len(dataset.classes)
model.fc = nn.Sequential(
    old_head,
    nn.Linear(num_features, num_classes)
)
model = model.to(device)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [15]:
from sklearn.metrics import accuracy_score

def evaluate_model(model, val_loader, device):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():  # No need to track gradients
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # Move to correct device
            outputs = model(images)  # Get model predictions
            _, preds = torch.max(outputs, 1)  # Get predicted class index

            all_labels.extend(labels.cpu().numpy())  # Store true labels
            all_preds.extend(preds.cpu().numpy())  # Store predictions

    accuracy = accuracy_score(all_labels, all_preds) * 100  
    print(f"Test Accuracy: {accuracy:.2f}%")
    
    return accuracy  
def train_model(model, train_loader,val_loader, optimizer, criterion, num_epochs, device):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
    
    
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
    
            optimizer.zero_grad()  # Clear gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            train_loss += loss.item()
            
        train_loss /= len(train_loader)

    
    
        val_acc=evaluate_model(model, val_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {val_acc:.2f}%")


In [16]:
for param in model.parameters():
    param.requires_grad = True

In [17]:
print("\n Training only the classifier (Feature Extraction Mode)...")
train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=20,device=device)


 Training only the classifier (Feature Extraction Mode)...
Test Accuracy: 36.02%
Epoch 1/20, Loss: 1.4328, Accuracy: 36.02%
Test Accuracy: 56.88%
Epoch 2/20, Loss: 1.0415, Accuracy: 56.88%
Test Accuracy: 60.98%
Epoch 3/20, Loss: 0.7180, Accuracy: 60.98%
Test Accuracy: 64.73%
Epoch 4/20, Loss: 0.9276, Accuracy: 64.73%
Test Accuracy: 68.71%
Epoch 5/20, Loss: 0.5423, Accuracy: 68.71%
Test Accuracy: 69.22%
Epoch 6/20, Loss: 0.5818, Accuracy: 69.22%
Test Accuracy: 72.34%
Epoch 7/20, Loss: 0.6233, Accuracy: 72.34%
Test Accuracy: 72.50%
Epoch 8/20, Loss: 0.8616, Accuracy: 72.50%
Test Accuracy: 74.57%
Epoch 9/20, Loss: 0.4754, Accuracy: 74.57%
Test Accuracy: 77.23%
Epoch 10/20, Loss: 0.4508, Accuracy: 77.23%
Test Accuracy: 80.90%
Epoch 11/20, Loss: 0.4296, Accuracy: 80.90%
Test Accuracy: 81.45%
Epoch 12/20, Loss: 0.4393, Accuracy: 81.45%
Test Accuracy: 83.09%
Epoch 13/20, Loss: 0.2887, Accuracy: 83.09%
Test Accuracy: 82.34%
Epoch 14/20, Loss: 0.4920, Accuracy: 82.34%
Test Accuracy: 83.36%
Epo

In [18]:
optimzer = optim.Adam(model.parameters(), lr=1e-6)
print("\n Training only the classifier (Feature Extraction Mode)...")
train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10,device=device)


 Training only the classifier (Feature Extraction Mode)...
Test Accuracy: 88.20%
Epoch 1/10, Loss: 0.2966, Accuracy: 88.20%
Test Accuracy: 90.94%
Epoch 2/10, Loss: 0.2291, Accuracy: 90.94%
Test Accuracy: 90.47%
Epoch 3/10, Loss: 0.1255, Accuracy: 90.47%
Test Accuracy: 90.20%
Epoch 4/10, Loss: 0.1423, Accuracy: 90.20%
Test Accuracy: 91.05%
Epoch 5/10, Loss: 0.0913, Accuracy: 91.05%
Test Accuracy: 89.77%
Epoch 6/10, Loss: 0.0822, Accuracy: 89.77%
Test Accuracy: 90.47%
Epoch 7/10, Loss: 0.2887, Accuracy: 90.47%
Test Accuracy: 92.19%
Epoch 8/10, Loss: 0.1164, Accuracy: 92.19%
Test Accuracy: 92.81%
Epoch 9/10, Loss: 0.0761, Accuracy: 92.81%
Test Accuracy: 92.62%
Epoch 10/10, Loss: 0.0445, Accuracy: 92.62%


In [22]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, precision_score, recall_score

model.eval()
all_labels = []
all_preds = []
val_loss = 0.0

with (torch.no_grad()):  # No need to track gradients for testing
    for images, labels in final_test_loader:
        images, labels = images.to(device), labels.to(device)  # Move labels and images to the current device
        outputs = model(images)  # Get model predictions
        _, preds = torch.max(outputs, 1)  # Get predicted class index (highest probability)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        all_labels.extend(labels.cpu().numpy())  # Store true labels
        all_preds.extend(preds.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(all_labels, all_preds)

# Compute F1-score
f1 = f1_score(all_labels, all_preds, average="weighted")  # Use "macro" for equal class weighting

precision = precision_score(all_labels, all_preds, average="weighted")  # Use "micro", "macro", or "weighted"

# Compute recall
recall = recall_score(all_labels, all_preds, average="weighted")
# Print results
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Loss: {val_loss:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# P
print("\nClassification Report:")
print(classification_report(all_labels, all_preds))


Test Accuracy: 90.59%
Test Loss: 50.4874
F1 Score: 0.9064
Precision: 0.9170
Recall: 0.9059

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       896
           1       1.00      1.00      1.00        64
           2       0.98      0.84      0.91      3200
           3       0.82      0.96      0.88      2240

    accuracy                           0.91      6400
   macro avg       0.93      0.95      0.94      6400
weighted avg       0.92      0.91      0.91      6400



In [24]:
torch.save(model,"coatnext_nano_rw_224_finetuned.sw_in1k.pth")

In [1]:
import timm

# List only models that have pretrained weights available
pretrained_models = timm.list_models(pretrained=False)

print(f"Pretrained models available: {len(pretrained_models)}")
for model_name in pretrained_models:
    print(model_name)


Pretrained models available: 1200
aimv2_1b_patch14_224
aimv2_1b_patch14_336
aimv2_1b_patch14_448
aimv2_3b_patch14_224
aimv2_3b_patch14_336
aimv2_3b_patch14_448
aimv2_huge_patch14_224
aimv2_huge_patch14_336
aimv2_huge_patch14_448
aimv2_large_patch14_224
aimv2_large_patch14_336
aimv2_large_patch14_448
bat_resnext26ts
beit_base_patch16_224
beit_base_patch16_384
beit_large_patch16_224
beit_large_patch16_384
beit_large_patch16_512
beitv2_base_patch16_224
beitv2_large_patch16_224
botnet26t_256
botnet50ts_256
caformer_b36
caformer_m36
caformer_s18
caformer_s36
cait_m36_384
cait_m48_448
cait_s24_224
cait_s24_384
cait_s36_384
cait_xs24_384
cait_xxs24_224
cait_xxs24_384
cait_xxs36_224
cait_xxs36_384
coat_lite_medium
coat_lite_medium_384
coat_lite_mini
coat_lite_small
coat_lite_tiny
coat_mini
coat_small
coat_tiny
coatnet_0_224
coatnet_0_rw_224
coatnet_1_224
coatnet_1_rw_224
coatnet_2_224
coatnet_2_rw_224
coatnet_3_224
coatnet_3_rw_224
coatnet_4_224
coatnet_5_224
coatnet_bn_0_rw_224
coatnet_nano_c